In [1]:
# Standard python library
import configparser
import sys
import os
import argparse
from collections import namedtuple

# Local resources
from gref4hsi.scripts import georeference
from gref4hsi.scripts import orthorectification
from gref4hsi.utils import parsing_utils, specim_parsing_utils
from gref4hsi.scripts import visualize
from gref4hsi.utils.config_utils import prepend_data_dir_to_relative_paths
from gref4hsi.utils.config_utils import customize_config

# Third party
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# From seabeepy/notebooks/flight_runner
import datetime as dt
import os
from pathlib import Path

from config import SETTINGS
from pyodm import Node
from subprocess import CalledProcessError

import seabeepy as sb


In [3]:


# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)



In [15]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/ntnu",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs/"

In [5]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2024-03-12 14:20:22.042685


In [6]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'capture' subdirectory, but NOT an 'orthophoto' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and not sb.ortho.check_subdir_exists(f.parent, "processed")
]

In [7]:
mission_list

[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_seaside-straumen_202402191358_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_hopavaagen_202402191311_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_hopavaagen_202402191253_ntnu_hyperspectral_74m')]

In [20]:
# Establish the ancillary data paths
from pathlib import Path

geoid_path = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/geoids/no_kv_HREF2018A_NN2000_EUREF89.tif')
config_template_path ='/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini'
lab_calibration_path = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')


PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')

In [30]:
# Establish the ancillary data paths
import yaml
import specim_process

specim_mission_folder = mission_list[0]

config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")

lab_calibration_path_local = os.path.join(temp_dir, "lab-calibration")


# Example copy from/to MINIO works
#sb.storage.copy_file('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini', 
#                      '/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim_copy.ini', minio_client, overwrite=False)
#sb.storage.copy_folder('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations', 
#                       '/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations_Copy', minio_client, overwrite=False)


# Example copying from MINIO to local WS
#sb.storage.copy_file('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini', 
#                     '/home/notebook/cogs/configuration_specim_copy.ini', minio_client, overwrite=False)

sb.storage.copy_folder('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations', 
                       '/home/notebook/cogs/Lab_Calibrations_Copy', minio_client, overwrite=False)


#specim_process.main(str(config_yaml), str(specim_mission_folder), geoid_path, config_template_path, lab_calibration_path)
    

    



OSError: Could not identify destination '/home/notebook/cogs/Lab_Calibrations_Copy' on MinIO.

In [17]:
# We need tu unveil how to read write the data to and from storage

local_copy_config = os.path.join('/home/notebook/config_specim.ini')


# Simple example:
sb.storage.copy_file(config_template_path, local_copy_config, minio_client, overwrite=False)




OSError: Could not identify destination '/home/notebook/config_specim.ini' on MinIO.

In [ ]:
import specim_process

# Process missions
for mission_fold in mission_list:
    mission_name = mission_fold.name
    print(f"\n################\nProcessing: {mission_name}")
    conf_yaml = os.path.join(mission_fold, "config.seabee.yaml")
    specim_process.main(conf_yaml, mission_fold)